In [ ]:
import os
import re
from pathlib import Path

def estandarizar_nombres_experimento(ruta_directorio, numero_exp, data_id=None, eliminar_fecha=True, simulacro=True):
    ruta = Path(ruta_directorio)
    
    if not ruta.exists():
        print(f"❌ Error: La ruta '{ruta}' no existe.")
        return

    print(f"--- Procesando: {ruta.name} (Simulacro: {simulacro}) ---")

    # 1. Patrón para FECHA
    regex_fecha_str = r'_\d{8}_\d{6}'
    patron_fecha = re.compile(regex_fecha_str)

    # 2. Patrón EXP + BASURA + CHUNK
    patron_con_chunk = re.compile(r'exp_\d+.*?_chunk')

    # 3. Patrón EXP + BASURA + FECHA
    patron_con_fecha = re.compile(r'exp_\d+.*?' + regex_fecha_str)

    # 4. Patrón EXP + RESTO (Corrección para tu caso)
    # Encuentra "exp_N" y todo el texto que le sigue hasta llegar al punto de la extensión.
    # [^.]* significa "cualquier cosa que no sea un punto".
    patron_exp_greedy = re.compile(r'exp_\d+[^.]*')

    # 5. Patrón simple (Solo números, por si acaso falla lo anterior)
    patron_exp_simple = re.compile(r'exp_\d+')

    archivos_modificados = 0

    for archivo in ruta.iterdir():
        if archivo.is_file():
            nombre_viejo = archivo.name
            nombre_nuevo = nombre_viejo

            # --- A. CASO: TENEMOS UN DATA_ID PARA INSERTAR ---
            if data_id:
                
                # PRIORIDAD 1: Es un archivo tipo CHUNK
                if 'chunk' in nombre_nuevo and patron_con_chunk.search(nombre_nuevo):
                    nombre_nuevo = patron_con_chunk.sub(
                        f'exp_{numero_exp}_{data_id}_chunk', 
                        nombre_nuevo
                    )

                # PRIORIDAD 2: No es chunk, pero TIENE FECHA
                elif patron_con_fecha.search(nombre_nuevo):
                    if eliminar_fecha:
                        nombre_nuevo = patron_con_fecha.sub(
                            f'exp_{numero_exp}_{data_id}', 
                            nombre_nuevo
                        )
                    else:
                        pass # Aquí podrías añadir lógica si decides no borrar fecha

                # PRIORIDAD 3: Limpieza completa (EL ARREGLO)
                # Si no tiene fecha ni es chunk, asumimos que todo lo que hay después 
                # del exp_N es el "data_id" viejo y queremos reemplazarlo por el nuevo.
                else:
                    # Usamos el patrón greedy que se "come" el sufijo viejo (ej: _simulation_testing)
                    nombre_nuevo = patron_exp_greedy.sub(
                        f'exp_{numero_exp}_{data_id}', 
                        nombre_nuevo
                    )

            # --- B. CASO: SOLO ACTUALIZAR NUMERO EXP (Sin Data_ID) ---
            else:
                 nombre_nuevo = patron_exp_simple.sub(f'exp_{numero_exp}', nombre_nuevo)

            # --- C. LIMPIEZA FINAL DE FECHA (Seguridad) ---
            if eliminar_fecha and patron_fecha.search(nombre_nuevo):
                 nombre_nuevo = patron_fecha.sub('', nombre_nuevo)

            # --- EJECUTAR ---
            if nombre_nuevo != nombre_viejo:
                if simulacro:
                    print(f"📝 [SIM] {nombre_viejo} \n       -> {nombre_nuevo}")
                else:
                    try:
                        archivo.rename(ruta / nombre_nuevo)
                        print(f"✅ [OK] {nombre_nuevo}")
                    except OSError as e:
                        print(f"❌ [Error] {e}")
                
                archivos_modificados += 1

    print(f"\nArchivos afectados: {archivos_modificados}")

In [39]:
# 1. Definir la ruta
script_path = os.getcwd()
project_path = os.path.join(script_path, '..', '..')

DATA_ID = 'simulation_7'
EXPERIMENT_ID = 'experiment_5'
folder_path = os.path.join(project_path, 'results', EXPERIMENT_ID, DATA_ID)

# 2. Ejecutar primero en modo SIMULACRO (recomendado)
# Esto cambiará 'exp_X' a 'exp_2' y borrará la fecha.
estandarizar_nombres_experimento(
    ruta_directorio=folder_path, 
    numero_exp=EXPERIMENT_ID.split('_')[-1], 
    data_id=DATA_ID,
    eliminar_fecha=True, 
    simulacro=False  # <--- Cambia a False cuando estés listo
)

--- Procesando: simulation_7 (Simulacro: False) ---
✅ [OK] mds_plot_exp_5_simulation_7.png
✅ [OK] plot_results_exp_5_simulation_7_acc.png
✅ [OK] results_exp_5_simulation_7.pkl
✅ [OK] results_exp_5_simulation_7_chunk_0.pkl
✅ [OK] results_exp_5_simulation_7_chunk_10.pkl
✅ [OK] results_exp_5_simulation_7_chunk_11.pkl
✅ [OK] results_exp_5_simulation_7_chunk_12.pkl
✅ [OK] results_exp_5_simulation_7_chunk_13.pkl
✅ [OK] results_exp_5_simulation_7_chunk_14.pkl
✅ [OK] results_exp_5_simulation_7_chunk_15.pkl
✅ [OK] results_exp_5_simulation_7_chunk_16.pkl
✅ [OK] results_exp_5_simulation_7_chunk_17.pkl
✅ [OK] results_exp_5_simulation_7_chunk_18.pkl
✅ [OK] results_exp_5_simulation_7_chunk_19.pkl
✅ [OK] results_exp_5_simulation_7_chunk_1.pkl
✅ [OK] results_exp_5_simulation_7_chunk_20.pkl
✅ [OK] results_exp_5_simulation_7_chunk_21.pkl
✅ [OK] results_exp_5_simulation_7_chunk_22.pkl
✅ [OK] results_exp_5_simulation_7_chunk_23.pkl
✅ [OK] results_exp_5_simulation_7_chunk_24.pkl
✅ [OK] results_exp_5_simula

---
---

In [28]:
import os
import re
import pickle
from pathlib import Path

def fusionar_exp4_en_exp5(ruta_directorio, data_id, simulacro=True):
    ruta = Path(ruta_directorio)
    
    if not ruta.exists():
        print(f"❌ Error: La ruta '{ruta}' no existe.")
        return

    print(f"--- Iniciando fusión de exp_4 en exp_5 (Simulacro: {simulacro}) ---")

    # Regex para extraer el número de chunk de los archivos exp_4 (si lo tienen)
    patron_chunk = re.compile(r'chunk_(\d+)')
    
    archivos_procesados = 0
    archivos_exp4 = list(ruta.glob('*exp_4*.pkl')) # Solo iteramos sobre los pkl

    for path_exp4 in archivos_exp4:
        nombre_exp4 = path_exp4.name
        
        # 1. Determinar el nombre del archivo exp_5 correspondiente
        match_chunk = patron_chunk.search(nombre_exp4)
        
        if match_chunk:
            num_chunk = match_chunk.group(1)
            nombre_exp5 = f'results_exp_5_{data_id}_chunk_{num_chunk}.pkl'
        else:
            nombre_exp5 = f'results_exp_5_{data_id}.pkl'
            
        path_exp5 = ruta / nombre_exp5

        # 2. Verificar que el archivo destino (exp_5) exista
        if not path_exp5.exists():
            print(f"⚠️ [Aviso] No se encontró el homólogo {nombre_exp5} para {nombre_exp4}. Se omite.")
            continue

        if simulacro:
            print(f"📝 [SIM] Se fusionará la data de: \n       {nombre_exp4} \n       -> DENTRO DE -> {nombre_exp5}\n       y se eliminará {nombre_exp4}\n")
            archivos_procesados += 1
            continue

        # 3. EJECUCIÓN REAL: Cargar, Fusionar, Guardar y Eliminar
        try:
            # Cargar datos de ambos pickles
            with open(path_exp4, 'rb') as f4, open(path_exp5, 'rb') as f5:
                data_4 = pickle.load(f4)
                data_5 = pickle.load(f5)

            # Lógica de fusión (Asumiendo diccionarios o listas)
            if isinstance(data_5, dict) and isinstance(data_4, dict):
                data_5.update(data_4) # Actualiza/Añade las keys de exp_4 en exp_5
            elif isinstance(data_5, list) and isinstance(data_4, list):
                data_5.extend(data_4) # Concatena las listas
            else:
                print(f"❌ [Error] Tipos de datos no soportados para fusión automática en {nombre_exp5}. (Son {type(data_5)} y {type(data_4)})")
                continue

            # Guardar el exp_5 actualizado
            with open(path_exp5, 'wb') as f5_out:
                pickle.dump(data_5, f5_out)
            
            # Eliminar el exp_4 viejo
            path_exp4.unlink()
            print(f"✅ [OK] Fusionado y eliminado: {nombre_exp4}")
            
            archivos_procesados += 1
            
        except Exception as e:
            print(f"❌ [Error crítico al procesar {nombre_exp4}]: {e}")

    print(f"\nOperación finalizada. Parejas procesadas: {archivos_procesados}")

In [37]:
DATA_ID = 'simulation_5'
EXPERIMENT_ID = 'experiment_5'
folder_path = os.path.join(project_path, 'results', EXPERIMENT_ID, DATA_ID)

fusionar_exp4_en_exp5(folder_path, data_id=DATA_ID, simulacro=False)

--- Iniciando fusión de exp_4 en exp_5 (Simulacro: False) ---
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_0_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_10_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_11_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_12_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_13_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_14_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_15_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_16_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_17_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_18_20250820_092759.pkl
✅ [OK] Fusionado y eliminado: results_exp_4_simu_6_chunk_19_20250820